In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [2]:
df_solar_new = pd.read_csv("D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
df_solar = pd.read_csv("D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")

In [3]:
def get_exact_time_lag_fast(df, value_column, timestamp_column='timestamp_utc', lag_hours=168):
    """
    Optimized version of get_exact_time_lag using vectorized operations.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the time series data
    value_column : str
        Name of the column containing values to be lagged
    timestamp_column : str
        Name of the column containing timestamps
    lag_hours : int
        Number of hours to look back for the lag
    
    Returns:
    --------
    pandas.Series
        Series containing the lagged values
    """
    # Calculate target timestamps
    target_timestamps = df[timestamp_column] - pd.Timedelta(hours=lag_hours)
    
    # Create a merged dataframe to find matches
    reference_df = pd.DataFrame({
        'reference_time': df[timestamp_column],
        'value': df[value_column]
    }).sort_values('reference_time')
    
    # Use searchsorted to find the insertion points
    idx = np.searchsorted(reference_df['reference_time'], target_timestamps, side='right') - 1
    
    # Handle cases where idx is -1 (target time before any reference time)
    idx = np.where(idx < 0, 0, idx)
    
    # Get the matched values
    return reference_df['value'].iloc[idx].values

In [4]:
df_solar_new.generation_mw = df_solar_new.generation_mw / 2

In [5]:
df_solar.drop(columns=['Unnamed: 0','boa_MWh','Wind_MW','Wind_MWh_credit'], inplace=True)
df_solar.valid_time = pd.to_datetime(df_solar.valid_time) 
df_solar.reference_time = pd.to_datetime(df_solar.reference_time)
df_solar = df_solar.groupby("valid_time").last().reset_index()

In [6]:
# Extracting hour of the day from 'valid_time'
df_solar['hour'] = df_solar['valid_time'].dt.hour

# Sine and cosine encoding for hour (for cyclical behavior)
df_solar['sin_hour'] = np.sin(2 * np.pi * df_solar['hour'] / 24)
df_solar['cos_hour'] = np.cos(2 * np.pi * df_solar['hour'] / 24)

# Day of the year (seasonality)
df_solar['day_of_year'] = df_solar['valid_time'].dt.dayofyear

# Sine and cosine encoding for day of the year (for cyclical seasonality)
df_solar['sin_day'] = np.sin(2 * np.pi * df_solar['day_of_year'] / 365)
df_solar['cos_day'] = np.cos(2 * np.pi * df_solar['day_of_year'] / 365)
df_solar['Mean_SolarRadiation_dwd'] = df_solar[[f'SolarDownwardRadiation_Point{i}_dwd' for i in range(20)]].mean(axis=1)
df_solar['Mean_Temperature_dwd'] = df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].mean(axis=1)
df_solar['Std_Temperature_dwd'] = df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].std(axis=1)
df_solar["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df_solar["Mean_SolarRadiation_dwd"].rolling(window= 1).mean()
df_solar["SolarDownwardRadiation_RW_dwd_Mean_1h"] = df_solar["Mean_SolarRadiation_dwd"].rolling(window= 2).mean()
# df_solar["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 1)
# df_solar["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 2)
# df_solar["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 48)
lag_configs = {
    "SolarDownwardRadiation_dwd_Mean_Lag_30min": ("Mean_SolarRadiation_dwd", 0.5),
    "SolarDownwardRadiation_dwd_Mean_Lag_1h": ("Mean_SolarRadiation_dwd", 1),
    "SolarDownwardRadiation_dwd_Mean_Lag_24h": ("Mean_SolarRadiation_dwd", 24),
}
for new_col, (source_col, hours) in lag_configs.items():
    df_solar[new_col] = get_exact_time_lag_fast(
        df_solar, 
        timestamp_column='valid_time',
        value_column=source_col,
        lag_hours=hours
    )

In [7]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [8]:
temperature_columns = [
 'Temperature_Point0_dwd',
 'Temperature_Point1_dwd',
 'Temperature_Point2_dwd',
 'Temperature_Point3_dwd',
 'Temperature_Point4_dwd',
 'Temperature_Point5_dwd',
 'Temperature_Point6_dwd',
 'Temperature_Point7_dwd',
 'Temperature_Point8_dwd',
 'Temperature_Point9_dwd',
 'Temperature_Point10_dwd',
 'Temperature_Point11_dwd',
 'Temperature_Point12_dwd',
 'Temperature_Point13_dwd',
 'Temperature_Point14_dwd',
 'Temperature_Point15_dwd',
 'Temperature_Point16_dwd',
 'Temperature_Point17_dwd',
 'Temperature_Point18_dwd',
 'Temperature_Point19_dwd',
]

irradiance_columns = [
    'SolarDownwardRadiation_Point0_dwd', 'SolarDownwardRadiation_Point1_dwd',
    'SolarDownwardRadiation_Point2_dwd', 'SolarDownwardRadiation_Point3_dwd',
    'SolarDownwardRadiation_Point4_dwd', 'SolarDownwardRadiation_Point5_dwd',
    'SolarDownwardRadiation_Point6_dwd', 'SolarDownwardRadiation_Point7_dwd',
    'SolarDownwardRadiation_Point8_dwd', 'SolarDownwardRadiation_Point9_dwd',
    'SolarDownwardRadiation_Point10_dwd', 'SolarDownwardRadiation_Point11_dwd',
    'SolarDownwardRadiation_Point12_dwd', 'SolarDownwardRadiation_Point13_dwd',
    'SolarDownwardRadiation_Point14_dwd', 'SolarDownwardRadiation_Point15_dwd',
    'SolarDownwardRadiation_Point16_dwd', 'SolarDownwardRadiation_Point17_dwd',
    'SolarDownwardRadiation_Point18_dwd', 'SolarDownwardRadiation_Point19_dwd',
]
for i in range(20):
    for source in ['dwd']:
        temp_col = f'Temperature_Point{i}_{source}'
        irradiance_col = f'SolarDownwardRadiation_Point{i}_{source}'
        panel_temp_col = f'Panel_Temperature_Point{i}_{source}'
        panel_eff_col = f'Panel_Efficiency_Point{i}_{source}'
        
        df_solar[panel_temp_col], df_solar[panel_eff_col] = zip(*df_solar.apply(
            lambda row: pv_temperature_efficiency(row[irradiance_col], row[temp_col]), axis=1))

In [9]:
df_solar["Panel_Temperature_dwd_mean"] = df_solar.filter(regex= r"Panel_Temperature.*_dwd").mean(axis= 1)
df_solar["Panel_Efficiency_dwd_mean"] = df_solar.filter(regex= r"Panel_Efficiency.*_dwd").mean(axis= 1)
df_solar["Panel_Temperature_dwd_std"] = df_solar.filter(regex= r"Panel_Temperature.*_dwd").std(axis= 1)
df_solar["Panel_Efficiency_dwd_std"] = df_solar.filter(regex= r"Panel_Efficiency.*_dwd").std(axis= 1)

In [10]:
df_solar_new.timestamp_utc = pd.to_datetime(df_solar_new.timestamp_utc) 
merged_df = pd.merge(df_solar_new, df_solar, left_on='timestamp_utc',right_on='valid_time', how='inner')

In [11]:
merged_df["Target_Capacity_MWP%"] = merged_df.generation_mw / merged_df.capacity_mwp

lag_configs = {
    "solar_mw_lag_48h": ("Solar_MWh_credit", 48),
    "capacity_mwp_lag_48h": ("capacity_mwp", 48),
    "Target_Capacity_MWP%_lag_48h": ("Target_Capacity_MWP%",48),
}
for new_col, (source_col, hours) in lag_configs.items():
    merged_df[new_col] = get_exact_time_lag_fast(
        merged_df, 
        timestamp_column='valid_time',
        value_column=source_col,
        lag_hours=hours)
    
merged_df.Solar_MWh_credit = merged_df.Solar_MWh_credit / merged_df.capacity_mwp
# merged_df["solar_mw_lag_48h"] = merged_df.Solar_MWh_credit.shift(periods= 96)
# merged_df["capacity_mwp_lag_48h"] = merged_df.capacity_mwp.shift(periods= 96)
# merged_df["Target_Capacity_MWP%_lag_48h"] = merged_df["Target_Capacity_MWP%"].shift(periods= 96)

In [12]:
merged_df = merged_df[(merged_df['timestamp_utc'] < '2022-11-21') | (merged_df['timestamp_utc'] > '2022-12-08')]

In [13]:
df_solar1 = merged_df[[ "timestamp_utc",
    "Mean_SolarRadiation_dwd",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_RW_dwd_Mean_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature_dwd",
    "Mean_Temperature_dwd",
    "cos_hour",
    "cos_day","solar_mw_lag_48h","capacity_mwp_lag_48h","Target_Capacity_MWP%_lag_48h",
    "Target_Capacity_MWP%","Solar_MWh_credit"]]

In [14]:
df_solar1 = df_solar1.dropna(axis=0)

In [15]:
df_solar1.capacity_mwp_lag_48h.mean()

2217.2152712806164

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# Assuming df_solar1 is already defined
# df_solar1 = ...

# Split the data into features and target
X = df_solar1.drop(columns=["Solar_MWh_credit","timestamp_utc","Target_Capacity_MWP%"]).values
y = df_solar1["Solar_MWh_credit"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# save the scaler
# import pickle
# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

In [42]:
# def pinball_loss(y_true, y_pred, quantiles):
#     """
#     Pinball loss for multiple quantiles.
    
#     Args:
#         y_true (Tensor): Ground truth values (batch_size, 1).
#         y_pred (Tensor): Predicted quantile values (batch_size, 9), where each column represents a quantile.
#         quantiles (list of floats): List of quantiles, length should be 9.
        
#     Returns:
#         Tensor: The average pinball loss for the batch.
#     """
#     loss = 0.0
#     for i, q in enumerate(quantiles):
#         errors = y_true - y_pred[:, i]
#         loss += torch.max((q - 1) * errors, q * errors).mean()
        
#     return loss / len(quantiles)


In [43]:
# def pinball_loss(y_true, y_pred, quantiles):
#     errors = y_true - y_pred
#     loss = torch.max((quantiles - 1) * errors, quantiles * errors)
#     return torch.mean(loss)

In [17]:
def pinball_loss(y_true, y_pred, quantiles):
    errors = y_true - y_pred

    # Ensure that quantiles is a tensor and expand it to match y_pred's shape
    quantiles = quantiles.view(1, -1).expand_as(y_pred)  # (batch_size, 9)

    # Compute pinball loss
    loss = torch.max((quantiles - 1) * errors, quantiles * errors)
    return torch.mean(loss)


In [18]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.3):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [20]:
# Previous imports and data preprocessing remain the same
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create TensorDataset and DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

# Create TensorDataset and DataLoader for testing
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)
# Hyperparameters
input_size = X_train_scaled.shape[1]  # Number of features
hidden_size = 128                     # Number of LSTM units
num_layers = 3                        # Number of LSTM layers
output_size = 9                       # Always 9 for 9 quantiles
dropout = 0.3                         # Dropout rate
learning_rate = 0.0001                # Learning rate for optimizer
batch_size = 32                       # Batch size
num_epochs = 500                      # Maximum number of epochs
patience = 10                         # Patience for early stopping
rel_improvement_threshold = 0.001      # Relative improvement threshold
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Quantiles (9 quantile levels)
quantiles = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype=torch.float32).to(device)

# Instantiate the model
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [21]:


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, 9) where 9 is the number of quantiles

        # Expand y_batch to match y_pred dimensions
        y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

        # Compute the loss
        loss = pinball_loss(y_batch_expanded, y_pred, quantiles)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Relative stopping
    rel_improvement = (previous_loss - epoch_loss) / previous_loss
    if rel_improvement < rel_improvement_threshold:
        print(f"Relative improvement below threshold. Stopping training.")
        break
    previous_loss = epoch_loss

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
model.load_state_dict(best_model)

# Test the model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, 9)

        # Expand y_batch to match y_pred dimensions
        y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

        # Compute the loss
        loss = pinball_loss(y_batch_expanded, y_pred, quantiles)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')

Epoch [1/500], Loss: 0.0076
Epoch [2/500], Loss: 0.0051
Epoch [3/500], Loss: 0.0049
Epoch [4/500], Loss: 0.0048
Epoch [5/500], Loss: 0.0048
Epoch [6/500], Loss: 0.0047
Epoch [7/500], Loss: 0.0047
Epoch [8/500], Loss: 0.0046
Epoch [9/500], Loss: 0.0046
Epoch [10/500], Loss: 0.0045
Epoch [11/500], Loss: 0.0045
Epoch [12/500], Loss: 0.0045
Epoch [13/500], Loss: 0.0044
Epoch [14/500], Loss: 0.0044
Epoch [15/500], Loss: 0.0044
Epoch [16/500], Loss: 0.0044
Epoch [17/500], Loss: 0.0043
Epoch [18/500], Loss: 0.0043
Epoch [19/500], Loss: 0.0043
Relative improvement below threshold. Stopping training.
Test Loss: 0.0077


In [23]:
X_test_tensor.shape

torch.Size([10026, 17])

In [24]:
#model eval on test
model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor.unsqueeze(1))
    final_test_loss = pinball_loss(y_test_tensor*2217.2046661135496, final_test_outputs*2217.2046661135496 , quantiles).item()
print(f"Final Test Loss: {final_test_loss:.4f}")

Final Test Loss: 17.0519


In [25]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [26]:
weather_df = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv')
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
weather_df.sort_values(by='ref_datetime', inplace=True)
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
weather_df.reset_index(inplace=True)

In [27]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

C:\Users\paulh\AppData\Local\Temp\ipykernel_23344\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_23344\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_23344\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_23344

In [28]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]

In [29]:
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()

In [30]:
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [31]:
def set_up_solar_features(df):
    df["hour"] = df.valid_datetime.dt.hour
    df["day_of_year"] = df.valid_datetime.dt.dayofyear
    df["cos_day_of_year"] = np.cos(2 * np.pi * df.day_of_year / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.hour / 24)
    df["Mean_SolarDownwardRadiation"] = df.SolarDownwardRadiation
    df["Mean_Temperature"] = df.Temperature
    df["Std_Temperature"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_Mean_30min"] = df.Mean_SolarDownwardRadiation.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1hour"] = df.Mean_SolarDownwardRadiation.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.Mean_SolarDownwardRadiation.shift(1)
    df["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df.Mean_SolarDownwardRadiation.shift(2)
    df["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df.Mean_SolarDownwardRadiation.shift(48)
    for i in range(len(distinct_lat_lon_pairs)):
        lat = distinct_lat_lon_pairs.latitude.iloc[i]
        lon = distinct_lat_lon_pairs.longitude.iloc[i]
        mask = (df_resampled_merged_solar.latitude == lat) & (df_resampled_merged_solar.longitude == lon)
        df[f"Temperature_{i}"] = pd.Series(df_resampled_merged_solar.Temperature[mask].values)[:len(df)]  # Fill gaps with NaN
        df[f"SolarDownwardRadiation_{i}"] = pd.Series(df_resampled_merged_solar.SolarDownwardRadiation[mask].values)[:len(df)]  # Fill gaps with NaN
    return df
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [32]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [33]:
for i in range(20):
    temp_col = f'Temperature_{i}'
    irradiance_col = f'SolarDownwardRadiation_{i}'
    panel_temp_col = f'Panel_Temperature_Point{i}'
    panel_eff_col = f'Panel_Efficiency_Point{i}'
    df_resampled_merged_solar2[panel_temp_col], df_resampled_merged_solar2[panel_eff_col] = pv_temperature_efficiency(df_resampled_merged_solar2[irradiance_col], df_resampled_merged_solar2[temp_col])

In [34]:
df_resampled_merged_solar2["Panel_Temperature_dwd_mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_dwd_mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Temperature_dwd_std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_dwd_std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").std(axis= 1)

In [35]:
df_resampled_merged_solar2["solar_mw_lag_48h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 96)
df_resampled_merged_solar2["capacity_mwp_lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)
df_resampled_merged_solar2["Target_Capacity_MWP%"] = df_resampled_merged_solar2.generation_mw / df_resampled_merged_solar2.capacity_mwp
df_resampled_merged_solar2["capacity_mwp_lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)
df_resampled_merged_solar2["Target_Capacity_MWP%_lag_48h"] = df_resampled_merged_solar2["Target_Capacity_MWP%"].shift(periods= 96)
df_resampled_merged_solar2.generation_mw = df_resampled_merged_solar2.generation_mw / df_resampled_merged_solar2.capacity_mwp

C:\Users\paulh\AppData\Local\Temp\ipykernel_23344\3718892184.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_resampled_merged_solar2["Target_Capacity_MWP%_lag_48h"] = df_resampled_merged_solar2["Target_Capacity_MWP%"].shift(periods= 96)


In [36]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "Mean_SolarDownwardRadiation",
    "SolarDownwardRadiation_RW_Mean_30min",
    "SolarDownwardRadiation_RW_Mean_1hour",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature",
    "Mean_Temperature",
    "cos_hour",
    "cos_day_of_year","solar_mw_lag_48h","capacity_mwp_lag_48h","Target_Capacity_MWP%_lag_48h",
    "generation_mw"]]
df_resampled_merged_solar3.dropna(inplace=True)

C:\Users\paulh\AppData\Local\Temp\ipykernel_23344\4222733903.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar3.dropna(inplace=True)


In [37]:
mean_to_multiply = df_resampled_merged_solar3.capacity_mwp_lag_48h.mean()
mean_to_multiply

2778.916828188406

In [38]:
X_newdata = df_resampled_merged_solar3.drop(columns=["generation_mw"])
Y = df_resampled_merged_solar3["generation_mw"]
X_newdata_scaled = scaler.transform(X_newdata)
X_newdata_tensor = torch.tensor(X_newdata_scaled, dtype=torch.float32)

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [39]:
model.eval()
with torch.no_grad():
    final_outputs = model(X_newdata_tensor.unsqueeze(1))
    final_outputs = final_outputs.numpy()
    final_outputs = final_outputs * mean_to_multiply

In [41]:
def pinball_loss(y_true, y_pred, quantiles):
    errors = y_true - y_pred
    loss = torch.max((quantiles - 1) * errors, quantiles * errors)
    return torch.mean(loss)

In [42]:
losses = []
for i in range(9):
    loss = pinball_loss(Y.values*mean_to_multiply, final_outputs[:, i], quantiles[i])
    losses.append(loss.item())
    print(f"Quantile {quantiles[i]} Loss: {loss.item()}")

Quantile 0.10000000149011612 Loss: 6.786298533657944
Quantile 0.20000000298023224 Loss: 12.761537509328067
Quantile 0.30000001192092896 Loss: 16.401998731687065
Quantile 0.4000000059604645 Loss: 19.484199929456555
Quantile 0.5 Loss: 21.316950776229636
Quantile 0.6000000238418579 Loss: 23.86955004791763
Quantile 0.699999988079071 Loss: 24.928852142495888
Quantile 0.800000011920929 Loss: 25.13722612900543
Quantile 0.8999999761581421 Loss: 26.779814005912606


In [43]:
np.mean(losses)

19.718491978410093

In [44]:
fig_test_api = go.Figure()
fig_test_api.add_trace(go.Scatter(y=Y.values*mean_to_multiply, name="True Test Api Values", mode="lines", line=dict(color="black")))
for i,loss in enumerate(losses):
    fig_test_api.add_trace(go.Scatter(y=final_outputs[:, i], name=f"Test Api Predictions ,Quantile {(i+1)*10}, Loss: {loss}", mode="lines"))
fig_test_api.update_layout(title=f"True Test Api Values vs Predictions", xaxis_title="Time", yaxis_title="Solar Production (MWh)")
fig_test_api.show()

In [47]:
# # Convert data to PyTorch tensors
# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# # Create TensorDataset and DataLoader for training
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

# # Create TensorDataset and DataLoader for testing
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# # Hyperparameters
# input_size = X_train_scaled.shape[1]  # Number of features
# hidden_size = 128                      # Number of LSTM units
# num_layers = 3                        # Number of LSTM layers
# output_size = 9                       # Always 9 for 9 quantiles
# dropout = 0.3                         # Dropout rate

# # Quantiles (9 quantile levels)
# # Quantiles (9 quantile levels)
# quantiles = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype=torch.float32).to(device)

# # Instantiate the model
# model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# # Loss function and optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.0001)

# # Training loop
# num_epochs = 500
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for X_batch, y_batch in train_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)

#         # Reshape input to add sequence length dimension
#         X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

#         # Forward pass
#         y_pred = model(X_batch)  # (batch_size, 9) where 9 is the number of quantiles

#         # Expand y_batch to match y_pred dimensions
#         y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

#         # Compute the loss
#         loss = pinball_loss(y_batch_expanded, y_pred, quantiles)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# # Test the model
# # Test the model
# model.eval()
# with torch.no_grad():
#     test_loss = 0.0
#     for X_batch, y_batch in test_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)

#         # Reshape input to add sequence length dimension
#         X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

#         # Forward pass
#         y_pred = model(X_batch)  # (batch_size, 9)

#         # Expand y_batch to match y_pred dimensions
#         y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

#         # Compute the loss
#         loss = pinball_loss(y_batch_expanded, y_pred, quantiles)
#         test_loss += loss.item()

#     print(f'Test Loss: {test_loss/len(test_loader):.4f}')



Epoch [1/500], Loss: 55.9958
Epoch [2/500], Loss: 53.1877
Epoch [3/500], Loss: 51.1088
Epoch [4/500], Loss: 49.2981
Epoch [5/500], Loss: 47.6409
Epoch [6/500], Loss: 46.1016
Epoch [7/500], Loss: 44.6490
Epoch [8/500], Loss: 43.2851
Epoch [9/500], Loss: 41.9725
Epoch [10/500], Loss: 40.7247
Epoch [11/500], Loss: 39.5110
Epoch [12/500], Loss: 38.3528
Epoch [13/500], Loss: 37.2532
Epoch [14/500], Loss: 36.2069
Epoch [15/500], Loss: 35.2084
Epoch [16/500], Loss: 34.2454
Epoch [17/500], Loss: 33.3293
Epoch [18/500], Loss: 32.4440
Epoch [19/500], Loss: 31.6012
Epoch [20/500], Loss: 30.7811
Epoch [21/500], Loss: 30.0083
Epoch [22/500], Loss: 29.2622
Epoch [23/500], Loss: 28.5301
Epoch [24/500], Loss: 27.8382
Epoch [25/500], Loss: 27.1874
Epoch [26/500], Loss: 26.5438
Epoch [27/500], Loss: 25.9397
Epoch [28/500], Loss: 25.3645
Epoch [29/500], Loss: 24.7996
Epoch [30/500], Loss: 24.2486
Epoch [31/500], Loss: 23.7352
Epoch [32/500], Loss: 23.2184
Epoch [33/500], Loss: 22.7494
Epoch [34/500], Los

In [63]:
#absolute error on test data use 50% quantile
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, 9)

        # Expand y_batch to match y_pred dimensions
        y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

        # Compute the loss
        loss = pinball_loss(y_batch_expanded, y_pred, quantiles)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')

Test Loss: 19.6510


In [77]:
print("Starting debug and predict function")
    
# Check if model is an instance of nn.Module
if not isinstance(model, torch.nn.Module):
    raise TypeError("The provided model is not an instance of torch.nn.Module")

print(f"Model type: {type(model)}")
print(f"Model architecture:\n{model}")

model.eval()  # Set the model to evaluation mode
print("Model set to evaluation mode")

with torch.no_grad():
    print("Entered no_grad context")
    
    # Check X_test type and convert if necessary
    if not isinstance(X_test, torch.Tensor):
        print("Converting X_test to torch.Tensor")
        try:
            X_test = torch.tensor(X_test, dtype=torch.float32)
        except Exception as e:
            print(f"Error converting X_test to tensor: {str(e)}")
            raise
    
    print(f"Original X_test shape: {X_test.shape}")
    print(f"X_test dtype: {X_test.dtype}")
    
    # Reshape X_test to 3D
    try:
        if X_test.dim() > 3:
            # Flatten all dimensions except the last one
            X_test = X_test.view(-1, X_test.size(-1))
        if X_test.dim() == 2:
            # Add a time dimension
            X_test = X_test.unsqueeze(1)
        print(f"Reshaped X_test. New shape: {X_test.shape}")
    except Exception as e:
        print(f"Error reshaping X_test: {str(e)}")
        raise
    
    # Move to the same device as the model
    try:
        device = next(model.parameters()).device
        X_test = X_test.to(device)
        print(f"Moved X_test to device: {device}")
    except Exception as e:
        print(f"Error moving X_test to device: {str(e)}")
        raise
    
    # Get predictions
    try:
        predictions = model(X_test)
        print(f"Predictions shape: {predictions.shape}")
        print(f"Sample predictions:\n{predictions[:5]}")
    except Exception as e:
        print(f"Error during model prediction: {str(e)}")
        raise

Starting debug and predict function
Model type: <class '__main__.LSTMPredictor'>
Model architecture:
LSTMPredictor(
  (lstm): LSTM(16, 128, num_layers=3, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=128, out_features=9, bias=True)
)
Model set to evaluation mode
Entered no_grad context
Original X_test shape: torch.Size([12284, 1, 16])
X_test dtype: torch.float32
Reshaped X_test. New shape: torch.Size([12284, 1, 16])
Moved X_test to device: cpu
Predictions shape: torch.Size([12284, 9])
Sample predictions:
tensor([[ 0.5242,  0.6697,  0.6944,  0.6196,  0.5333,  0.3655,  0.1972, -0.0841,
         -0.2925],
        [ 0.5240,  0.6694,  0.6941,  0.6193,  0.5330,  0.3653,  0.1970, -0.0843,
         -0.2926],
        [ 0.5238,  0.6693,  0.6940,  0.6192,  0.5329,  0.3652,  0.1969, -0.0844,
         -0.2926],
        [ 0.5233,  0.6687,  0.6933,  0.6185,  0.5323,  0.3646,  0.1964, -0.0847,
         -0.2928],
        [ 0.5234,  0.6688,  0.6935,  0.6187,  0.5324,  0.3647,  0.1965, -0.084

In [ ]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
        # Ensure X_test is a tensor
        if not isinstance(X_test, torch.Tensor):
            X_test = torch.tensor(X_test, dtype=torch.float32)
        
        # Move to the same device as the model
        X_test = X_test.to(next(model.parameters()).device)
        
        # Reshape input to add sequence length dimension
        X_test = X_test.unsqueeze(1)  # Shape becomes (batch_size, 1, input_size)
        
        # Get predictions
        predictions = model(X_test)

In [78]:
predictions = predictions.cpu().numpy()

In [79]:
predictions_50 = predictions[:, 4]  # 50th quantile

In [80]:
# Plot the predictions vs. true targets with Plotly
y_pred_numpy = predictions_50
y_test_numpy = y_test

fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.flatten(), y=y_pred_numpy.flatten(), mode='markers', name='Predictions'))

fig.update_layout(title='Predictions vs. True Targets',
                  xaxis_title='True Targets',
                  yaxis_title='Predictions')

fig.show()

In [81]:
np.mean(np.abs(y_pred_numpy.flatten() - y_test_numpy.flatten()))

61.14033973955489

In [82]:
mae = np.mean(np.abs(y_test - predictions_50))
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Create scatter plot (y_true vs y_pred)
fig = go.Figure()

# Add scatter plot for true vs predicted values
fig.add_trace(go.Scatter(x=y_test, 
                         y=predictions_50, 
                         mode='markers', 
                         name='Predicted vs True'))

# Add a line y = x for reference (perfect prediction line)
fig.add_trace(go.Scatter(x=y_test, 
                         y=y_test, 
                         mode='lines', 
                         name='y = x', 
                         line=dict(color='red', dash='dash')))

# Update layout
fig.update_layout(title='Scatter Plot: True Values vs Predicted (50% Quantile)',
                  xaxis_title='True Values',
                  yaxis_title='Predicted Values',
                  template='plotly')

# Show the plot
fig.show()

Mean Absolute Error (MAE): 61.1403
